In [ ]:
#Web Scraping y filtro para Minciencias y separacion de productos

In [ ]:
#pip install openpyxl

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from scrapy.selector import Selector
import requests
from bs4 import BeautifulSoup
import warnings

In [2]:
warnings.filterwarnings('ignore')
warnings.warn('InsecureRequestWarning')
warnings.warn('Do not show this message')

In [3]:
dfgrupos = pd.read_excel("BDRamcaRed.xlsx", engine='openpyxl') #, sheet_name='Sheet1'
tamaño_dfgrupos =dfgrupos.shape
url= dfgrupos["LINKS"]
dfcon = pd.DataFrame()

In [4]:
url

0     https://scienti.minciencias.gov.co/gruplac/jsp...
1     https://scienti.minciencias.gov.co/gruplac/jsp...
2     https://scienti.minciencias.gov.co/gruplac/jsp...
3     https://scienti.minciencias.gov.co/gruplac/jsp...
4     https://scienti.minciencias.gov.co/gruplac/jsp...
5     https://scienti.minciencias.gov.co/gruplac/jsp...
6     https://scienti.minciencias.gov.co/gruplac/jsp...
7     https://scienti.minciencias.gov.co/gruplac/jsp...
8     https://scienti.minciencias.gov.co/gruplac/jsp...
9     https://scienti.minciencias.gov.co/gruplac/jsp...
10    https://scienti.minciencias.gov.co/gruplac/jsp...
11    https://scienti.minciencias.gov.co/gruplac/jsp...
12    https://scienti.minciencias.gov.co/gruplac/jsp...
13    https://scienti.minciencias.gov.co/gruplac/jsp...
14    https://scienti.minciencias.gov.co/gruplac/jsp...
15    https://scienti.minciencias.gov.co/gruplac/jsp...
16    https://scienti.minciencias.gov.co/gruplac/jsp...
17    https://scienti.minciencias.gov.co/gruplac

In [6]:
#Para el ejemplo 
#url = url[0:5]

In [20]:
#lo use porque se corto en el link 8 

dfcon = pd.read_excel("InormacionFiltrada.xlsx",engine='openpyxl') #sheet_name='Sheet1', 
dfcon

,Index,Unnamed: 0,Nombre del Grupo,Datos básicos,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo,PRODUCCIÓN DE FORMACIÓN Y EXTENSIÓN,Programa académico de doctorado,...,Participación Ciudadana en Proyectos de CTI,"Producción en arte, arquitectura y diseño",ACTIVIDADES DE FORMACIÓN,Asesorías al Programa Ondas,Curso de Corta Duración Dictados,Trabajos dirigidos/turorías,ACTIVIDADES COMO EVALUADOR,Jurado/Comisiones evaluadoras de trabajo de grado,Participación en comités de evaluación,Demás trabajos
0,0,0.0,INAM-USTA (Ingeniería Ambiental - Universidad ...,Año y mes de formación: 2010 - 1... \n Depar...,1.- UNIVERSIDAD SANTO TOMAS - (Avalado),Plan de trabajo: Las propuestas de investigaci...,1.- Economía y Gestión Ambiental \n 2.- Ordena...,Nombre \n 1.- Angela María Jaramillo Londoño \...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,Grupo de Ingeniería y Gestión Ambiental GIGA,Año y mes de formación: 1992 - 1... \n Depar...,1.- UNIVERSIDAD DE ANTIOQUIA - (Avalado),Plan de trabajo: -Apoyar la capacitación de lo...,1.- Calidad del Aire \n 2.- Calidad del agua \...,Nombre \n 1.- Juan Fernando Salazar Villegas \...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,CENTRO DE INVESTIGACIONES EN INGENIERIA AMBIENTAL,Año y mes de formación: 1991 - 6... \n Depar...,1.- UNIVERSIDAD DE LOS ANDES - (Avalado),Plan de trabajo: Estado del arte: Objetivos: C...,1.- CALIDAD AMBIENTAL \n 2.- CONTROL DE LA CON...,Nombre \n 1.- Ricardo Morales Betancourt \n 2....,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,GRUPO DE INVESTIGACIONES AMBIENTALES,Año y mes de formación: 1974 - 1... \n Depar...,1.- UNIVERSIDAD PONTIFICIA BOLIVARIANA - (Aval...,Plan de trabajo: - A través de la capacitación...,1.- LABORATORIO AMBIENTAL \n 2.- LINEA DE ESTU...,Nombre \n 1.- MARILUZ BETANCUR VELEZ \n 2.- Al...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,Instituto de Estudios Hidráulicos y Ambientale...,Año y mes de formación: 1998 - 8... \n Depar...,1.- FUNDACIÓN UNIVERSIDAD DEL NORTE - (Avalado),Plan de trabajo: El Instituto de Estudios Hidr...,1.- Ingeniería e Hidráulica Fluvial y Costera ...,Nombre \n 1.- Aymer Yeferson MATURANA CORDOBA ...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5.0,"Grupo de investigación en Nanotecnología, Bioi...",Año y mes de formación: 1996 - 7... \n Depar...,1.- UNIVERSIDAD CENTRAL - (Avalado) \n 2.- KUS...,Plan de trabajo: Generar y gestionar conocimie...,"1.- Bioingeniería en salud, ambiente y agro \n...",Nombre \n 1.- Olga Yaneth Vásquez Ochoa \n 2.-...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,6.0,GRUPO DE INVESTIGACIÓN EN USO y MANEJO DE SUEL...,Año y mes de formación: 1995 - 7... \n Depar...,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: El programa de la Investigaci...,1.- BIOLOGÍA DEL SUELO \n 2.- EVALUACION Y DEG...,Nombre \n 1.- Juan Carlos Menjivar Flores \n 2...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,7.0,Química Ambiental y Computacional,Año y mes de formación: 1995 - 1... \n Depar...,1.- UNIVERSIDAD DE CARTAGENA - (Avalado),Plan de trabajo: El Grupo de Química Ambiental...,1.- Ambiente y Desarrollo Sostenible \n 2.- Bi...,Nombre \n 1.- Jesús Tadeo Olivero Verbel \n 2....,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,8.0,OCEANICOS - Grupo de Oceanografía e Ingeniería...,Año y mes de formación: 2002 - 9... \n Depar...,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: El grupo de Oceanografía e In...,1.- Clima \n 2.- Ecosistemas Marinos \n 3.- En...,Nombre \n 1.- Andrés Fernando Osorio Arias \n ...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,9.0,Posgrado en Aprovechamiento de Recursos Hidráu...,Año y mes de formación: 1984 - 1... \n Depar...,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: Se Espera impulsar de manera ...,1.- Aguas Subterráneas \n 2.- Calidad del Agua...,Nombre \n 1.- Andrés Ochoa Jaramillo \n

In [47]:
palabras_filtro = "agua"
palabras_filtro2 = "monitoreo"


####OJO EL CICLO### 
for i in range (0,tamaño_dfgrupos [0]): #tamaño_dfgrupos [0]
    count = i
    try:
        print("Procesando link  ",count," .")
        url = dfgrupos["LINKS"][i]
        r = requests.get(url, verify=False)
        soup = BeautifulSoup(r.text, 'lxml')
        NomGrupo = soup.find("span", class_='celdaEncabezado')
        NombreGrupo = NomGrupo.text
        #Filtrar todas las tablas
        table = soup.find_all('table')
        df = pd.read_html(str(table))
        #Numero de de tablas 
        NumTablas = len(df)
        #CICLO 
        dfcon1 = pd.DataFrame()
        dfcon1["Index"] = [i]
        dfcon1["Nombre del Grupo"] = [NombreGrupo]
        #Tabla numero 0, datos basicos, que es diferente 
        #Tabla de datos 
        df = pd.read_html(str(table))[0]
        #Imputacion de los NAN
        df= df.fillna('Sin dato')
        #Tabla 1 
        tamaño = df.shape
        Nombre = df[0][0]
        Dato = []
        for j in range(1,tamaño[0]):
            Dato.append(df[0][j]+":  "+df[1][j]+"... ")
        Datos = " \n ".join(Dato)
        Datos
        dfcon1[Nombre]= [Datos]
        # Siguientes Tablas 1-5
        print("Procesando link  ",count," ..")
        for i in range (1,6):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]):
                Datos.append(df[0][j])
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos] 
        print("Procesando link  ",count," ...")
        # Siguientes Tablas 6-12
        for i in range (6,13):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            df_filtrado = df[df[0].str.contains(palabras_filtro,case=False) & df[0].str.contains(palabras_filtro2,case=False) ] 
            Datos = []
            for j in range(0,tamaño[0]):
                try:
                    Datos.append(df_filtrado[0][j])
                except:
                    pass
            Datos = " \n ".join(Datos) 
            dfcon1[Nombre]= [Datos]
        print("Procesando link  ",count," ....")
        #Tabla 13 -44
        for i in range (13,45):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]):
                try:
                    df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False) ]               
                    Datos.append(df_filtrado[1][j])
                except:
                    pass
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos]
        print("Procesando link  ",count," .....")
        #Table 45 - 46
        for i in range (45,47):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]-1):
                df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False) ]
                Datos.append(df[1][j])
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos]
        print("Procesando link  ",count," ......")
        #Tabla 47 - 84 
        for i in range (47,84):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]-1):
                try:
                    df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False)] #Case = false para que no importen mayusculas
                    Datos.append(df_filtrado[1][j])
                except:
                    pass
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos] 
        dfcon = pd.concat([dfcon, dfcon1])
    except:
            print ("error en link  ",count)
dfcon = dfcon.set_index("Index")
dfcon

Procesando link   0  .
error en link   0
Procesando link   1  .
error en link   1
Procesando link   2  .
error en link   2
Procesando link   3  .
error en link   3
Procesando link   4  .
error en link   4
Procesando link   5  .
error en link   5
Procesando link   6  .
error en link   6
Procesando link   7  .
error en link   7
Procesando link   8  .
error en link   8
Procesando link   9  .
error en link   9
Procesando link   10  .
error en link   10
Procesando link   11  .
error en link   11
Procesando link   12  .
error en link   12
Procesando link   13  .
error en link   13
Procesando link   14  .
error en link   14
Procesando link   15  .
error en link   15
Procesando link   16  .
error en link   16
Procesando link   17  .
error en link   17
Procesando link   18  .
error en link   18
Procesando link   19  .
error en link   19
Procesando link   20  .
error en link   20
Procesando link   21  .
error en link   21
Procesando link   22  .
error en link   22
Procesando link   23  .
error e

KeyError: "None of ['Index'] are in the columns"

In [48]:
dfcon.to_excel("InormacionFiltrada.xlsx")

In [49]:
df = pd.read_excel("InormacionFiltrada.xlsx",engine='openpyxl') #sheet_name='Sheet1', 
df = df.set_index("Index")

In [50]:
df.columns

Index(['Unnamed: 0', 'Nombre del Grupo', 'Datos básicos', 'Instituciones',
       'Plan Estratégico', 'Líneas de investigación declaradas por el grupo',
       'Integrantes del grupo', 'PRODUCCIÓN DE FORMACIÓN Y EXTENSIÓN',
       'Programa académico de doctorado', 'Programa académico de maestría',
       'Otro programa académico', 'Curso de doctorado', 'Curso de maestría',
       'Curso especializado de extensión', 'PRODUCCIÓN BIBLIOGRÁFICA',
       'Artículos publicados', 'Libros publicados',
       'Traducciones Filológicas y Edición de Fuentes',
       'Capítulos de libro publicados', 'Documentos de trabajo',
       'Otra publicación divulgativa', 'Otros artículos publicados',
       'Libros de formación',
       'Libros de divulgación y/o Compilación de divulgación',
       'Manuales y Guías Especializadas', 'Otros Libros publicados',
       'Traducciones', 'Notas científicas', 'PRODUCCIÓN TÉCNICA Y TECNOLÓGICA',
       'Cartas, mapas o similares', 'Conceptos técnicos',
       'Di

In [51]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,Nombre del Grupo,Datos básicos,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo,PRODUCCIÓN DE FORMACIÓN Y EXTENSIÓN,Programa académico de doctorado,Programa académico de maestría,Otro programa académico,...,Participación Ciudadana en Proyectos de CTI,"Producción en arte, arquitectura y diseño",ACTIVIDADES DE FORMACIÓN,Asesorías al Programa Ondas,Curso de Corta Duración Dictados,Trabajos dirigidos/turorías,ACTIVIDADES COMO EVALUADOR,Jurado/Comisiones evaluadoras de trabajo de grado,Participación en comités de evaluación,Demás trabajos
Index,,,,,,,,,,,,,,,,,,,,,
0,INAM-USTA (Ingeniería Ambiental - Universidad ...,Año y mes de formación: 2010 - 1... \n Depar...,1.- UNIVERSIDAD SANTO TOMAS - (Avalado),Plan de trabajo: Las propuestas de investigaci...,1.- Economía y Gestión Ambiental \n 2.- Ordena...,Nombre \n 1.- Angela María Jaramillo Londoño \...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grupo de Ingeniería y Gestión Ambiental GIGA,Año y mes de formación: 1992 - 1... \n Depar...,1.- UNIVERSIDAD DE ANTIOQUIA - (Avalado),Plan de trabajo: -Apoyar la capacitación de lo...,1.- Calidad del Aire \n 2.- Calidad del agua \...,Nombre \n 1.- Juan Fernando Salazar Villegas \...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CENTRO DE INVESTIGACIONES EN INGENIERIA AMBIENTAL,Año y mes de formación: 1991 - 6... \n Depar...,1.- UNIVERSIDAD DE LOS ANDES - (Avalado),Plan de trabajo: Estado del arte: Objetivos: C...,1.- CALIDAD AMBIENTAL \n 2.- CONTROL DE LA CON...,Nombre \n 1.- Ricardo Morales Betancourt \n 2....,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GRUPO DE INVESTIGACIONES AMBIENTALES,Año y mes de formación: 1974 - 1... \n Depar...,1.- UNIVERSIDAD PONTIFICIA BOLIVARIANA - (Aval...,Plan de trabajo: - A través de la capacitación...,1.- LABORATORIO AMBIENTAL \n 2.- LINEA DE ESTU...,Nombre \n 1.- MARILUZ BETANCUR VELEZ \n 2.- Al...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Instituto de Estudios Hidráulicos y Ambientale...,Año y mes de formación: 1998 - 8... \n Depar...,1.- FUNDACIÓN UNIVERSIDAD DEL NORTE - (Avalado),Plan de trabajo: El Instituto de Estudios Hidr...,1.- Ingeniería e Hidráulica Fluvial y Costera ...,Nombre \n 1.- Aymer Yeferson MATURANA CORDOBA ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,PICHIHUEL,Año y mes de formación: 1996 - 2... \n Depar...,1.- UNIVERSIDAD DE LA GUAJIRA - (Avalado),Plan de trabajo: - FORTALECIMIENTO DEL GRUPO *...,1.- Biologia productiva \n 2.- Gestión integra...,Nombre \n 1.- Jairo Rafael Rosado Vega \n 2.- ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Grupo de Estudios Ambientales,Año y mes de formación: 2000 - 10... \n Depa...,1.- UNIVERSIDAD TECNOLOGICA DEL CHOCO DIEGO LU...,Plan de trabajo: GRUPO DE ESTUDIOS AMBIENTALES...,1.- Calidad de agua \n 2.- Comunidad y Ambient...,Nombre \n 1.- Jhon Jairo Lopez Perea \n 2.- He...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,69.- Trabajos dirigidos/Tutorías de otro tipo ...,NaN,40.- Pregrado : SEGUIMIENTO MONITOREO Y CONTRO...,NaN,NaN
41,Grupo de investigación en Agua y Medio Ambient...,Año y mes de formación: 1996 - 10... \n Depa...,1.- CORPORACIÓN CENTRO DE CIENCIA Y TECNOLOGÍA...,Plan de trabajo: El plan de trabajo del grupo ...,1.- Dinámicas ambientales y territoriales \n 2...,Nombre \n 1.- Carolina María Rodríguez Ortiz \...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df.to_excel("InormacionFiltrada.xlsx")

In [53]:
#leer el DataFrame e imprimir cada columna como una tabla nueva

In [54]:
##Eliminar los sin dato y los vacios (:  -)

#Crear tabla de datos basicos
DatosBasicos = pd.DataFrame(columns=['Nombre del Grupo','Datos básicos'])
for j in range(0,df.shape[0]):
    try:
        dfDatosBasidos = pd.DataFrame(columns=['Nombre del Grupo','Datos básicos'])
        ListaDatosBasicos = df["Datos básicos"][j].split("... ")
        for i in range (0,len(ListaDatosBasicos)-1):
            if i == 0:
                dfDatosBasidos.loc[len(dfDatosBasidos)] = [df["Nombre del Grupo"][j], ListaDatosBasicos[i]]
            else:
                dfDatosBasidos.loc[len(dfDatosBasidos)] = ["", ListaDatosBasicos[i]]
        #REMOVER LAS FILAS QUE NO SIRVEN 
        v1 = dfDatosBasidos.iloc[:, 1].str.contains("sin dato",case=False)
        v2 = dfDatosBasidos.iloc[:, 1].str.contains(":  -",case=False)
        for i in range (0,len(ListaDatosBasicos)-1):
            if (v1[i] == True) or (v2[i] == True):
                dfDatosBasidos = dfDatosBasidos.drop([i],axis=0)
        DatosBasicos = pd.concat([DatosBasicos, dfDatosBasidos])
    except: 
        pass
DatosBasicos.to_excel("DatosBasicos.xlsx")
DatosBasicos

,Nombre del Grupo,Datos básicos


In [55]:
# Crear tabla de informacion de los grupos 
DatosGrupo = df.iloc[:,0:6].drop(["Datos básicos"],axis=1)
DatosGrupo.to_excel("DatosGrupo.xlsx")
DatosGrupo

,Nombre del Grupo,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo
Index,,,,,
0,INAM-USTA (Ingeniería Ambiental - Universidad ...,1.- UNIVERSIDAD SANTO TOMAS - (Avalado),Plan de trabajo: Las propuestas de investigaci...,1.- Economía y Gestión Ambiental \n 2.- Ordena...,Nombre \n 1.- Angela María Jaramillo Londoño \...
1,Grupo de Ingeniería y Gestión Ambiental GIGA,1.- UNIVERSIDAD DE ANTIOQUIA - (Avalado),Plan de trabajo: -Apoyar la capacitación de lo...,1.- Calidad del Aire \n 2.- Calidad del agua \...,Nombre \n 1.- Juan Fernando Salazar Villegas \...
2,CENTRO DE INVESTIGACIONES EN INGENIERIA AMBIENTAL,1.- UNIVERSIDAD DE LOS ANDES - (Avalado),Plan de trabajo: Estado del arte: Objetivos: C...,1.- CALIDAD AMBIENTAL \n 2.- CONTROL DE LA CON...,Nombre \n 1.- Ricardo Morales Betancourt \n 2....
3,GRUPO DE INVESTIGACIONES AMBIENTALES,1.- UNIVERSIDAD PONTIFICIA BOLIVARIANA - (Aval...,Plan de trabajo: - A través de la capacitación...,1.- LABORATORIO AMBIENTAL \n 2.- LINEA DE ESTU...,Nombre \n 1.- MARILUZ BETANCUR VELEZ \n 2.- Al...
4,Instituto de Estudios Hidráulicos y Ambientale...,1.- FUNDACIÓN UNIVERSIDAD DEL NORTE - (Avalado),Plan de trabajo: El Instituto de Estudios Hidr...,1.- Ingeniería e Hidráulica Fluvial y Costera ...,Nombre \n 1.- Aymer Yeferson MATURANA CORDOBA ...
...,...,...,...,...,...
39,PICHIHUEL,1.- UNIVERSIDAD DE LA GUAJIRA - (Avalado),Plan de trabajo: - FORTALECIMIENTO DEL GRUPO *...,1.- Biologia productiva \n 2.- Gestión integra...,Nombre \n 1.- Jairo Rafael Rosado Vega \n 2.- ...
40,Grupo de Estudios Ambientales,1.- UNIVERSIDAD TECNOLOGICA DEL CHOCO DIEGO LU...,Plan de trabajo: GRUPO DE ESTUDIOS AMBIENTALES...,1.- Calidad de agua \n 2.- Comunidad y Ambient...,Nombre \n 1.- Jhon Jairo Lopez Perea \n 2.- He...
41,Grupo de investigación en Agua y Medio Ambient...,1.- CORPORACIÓN CENTRO DE CIENCIA Y TECNOLOGÍA...,Plan de trabajo: El plan de trabajo del grupo ...,1.- Dinámicas ambientales y territoriales \n 2...,Nombre \n 1.- Carolina María Rodríguez Ortiz \...


In [56]:
# Crear todas las tablas en archivos diferentes. 

df1 = df.set_index("Nombre del Grupo")
for i in range(5,84):
    try:
        NombreColumna = df1.columns[i]
        Nombre = str(i)+' '+NombreColumna+".xlsx"
        dfsin_nan = df1[NombreColumna].dropna()
        try:
            if type(dfsin_nan[0]) != str():
                dfsin_nan.to_excel(Nombre)
        except:
            pass
    except:
        pass